In [1]:
from selenium import webdriver # 1 login 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
import time

driver = webdriver.Chrome()
driver.get("https://az3.ondemand.esker.com/ondemand/webaccess/asf/home.aspx")
driver.maximize_window()
time.sleep(1)

driver.find_element(By.XPATH, '//*[@id="ctl03_tbUser"]').send_keys("john.tan@sh-cogent.com.sg")
driver.find_element(By.XPATH, '//*[@id="ctl03_tbPassword"]').send_keys("Esker3838")
driver.find_element(By.XPATH, '//*[@id="ctl03_btnSubmitLogin"]').click()
time.sleep(5)

def hover(driver, x_path):
    elem_to_hover = driver.find_element(By.XPATH, x_path)
    hover = ActionChains(driver).move_to_element(elem_to_hover)
    hover.perform()

time.sleep(3)
x_path_hover = '//*[@id="mainMenuBar"]/td/table/tbody/tr/td[36]/a/div' #arrow
hover(driver, x_path_hover)
time.sleep(2)


In [ ]:
from pathlib import Path
import re
import pandas as pd

# ---------------------------
# Iterate through log files and parse vendor data from 'Data:' sections
# Handles fixed-width spacing and odd lines where vendor_number is glued to the name
# ---------------------------

dir_path = r"C:/Users/john.tan/Documents/power_apps_esker_vendor/esker_vendor_update/Log/"
def parse_vendor_data_from_logs(dir_path: str = dir_path,
                                recursive: bool = True,
                                keep: str = 'first',
                                pattern: str = "log_2025.txt",
                                ) -> pd.DataFrame:
    """
    Walk `dir_path`, read all .txt files, extract the 'Data:' block rows with
    columns [company_code, vendor_number, name], de-duplicate, and return a DataFrame.

    - Handles fixed-width spacing and odd lines where vendor_number is glued to the name
      (e.g., '1000265996AARDWOLF...').
    - Ignores other sections like 'Process initialized/completed'.
    """
    base = Path(dir_path)
    rows = []

    def parse_data_row(line: str):
        s = line.rstrip("\n")
        if not s.strip():
            return None

        # 1) Typical fixed-width logs: split on 2+ spaces
        parts = re.split(r"\s{2,}", s.strip())
        if len(parts) >= 3:
            company_code = parts[0].strip()
            vendor_number = parts[1].strip()
            name = " ".join(parts[2:]).strip()

            # If vendor_number accidentally includes the start of the name, split it
            m2 = re.match(r"^(\d{5,})([A-Za-z].*)$", vendor_number)
            if m2:
                vendor_number = m2.group(1)
                name = (m2.group(2) + (" " + name if name else "")).strip()
            return company_code, vendor_number, name

        # 2) Fallback: token, digits, then glued name
        m = re.match(r"^\s*(\S+)\s+(\d{5,})([A-Za-z].+)$", s)
        if m:
            return m.group(1), m.group(2), m.group(3).strip()

        # 3) Very loose: token, token, rest (and split leading digits if present)
        m = re.match(r"^\s*(\S+)\s+(\S+)\s+(.+)$", s)
        if m:
            company_code, vendor_number, name = m.group(1), m.group(2), m.group(3).strip()
            m3 = re.match(r"^(\d{5,})([A-Za-z].*)$", vendor_number)
            if m3:
                vendor_number = m3.group(1)
                name = (m3.group(2) + " " + name).strip()
            return company_code, vendor_number, name

        return None

    for fp in base.rglob("*.txt"):
        try:
            lines = fp.read_text(encoding="utf-8", errors="ignore").splitlines()
        except Exception:
            continue  # skip unreadable files

        i, n = 0, len(lines)
        while i < n:
            if lines[i].strip().startswith("Data:"):
                # Skip to header (next non-empty line)
                i += 1
                while i < n and not lines[i].strip():
                    i += 1
                # header line present but we don't need to parse it strictly
                i += 1
                # Read data rows until blank/new section
                while i < n:
                    cur = lines[i]
                    cur_stripped = cur.strip()
                    if (not cur_stripped) or cur_stripped.startswith(("Process completed", "Process initialized", "Updated", "Data:")):
                        break
                    # Skip accidental header repeats
                    if {"company_code", "vendor_number", "name"}.issubset(set(re.split(r"\s+", cur_stripped))):
                        i += 1
                        continue
                    parsed = parse_data_row(cur)
                    if parsed:
                        rows.append(parsed)
                    i += 1
            else:
                i += 1

    df = pd.DataFrame(rows, columns=["company_code", "vendor_number", "name"]) if rows else pd.DataFrame(columns=["company_code", "vendor_number", "name"])
    for col in df.columns:
        df[col] = df[col].astype(str).str.strip()
    

    #write to csv for checking and persistent storage
    with open(dir_path + 'parsed_vendor_data.csv', 'w', newline='', encoding='utf-8') as f:
        df.drop_duplicates().reset_index(drop=True)
        df.to_csv(f, index=False)
    return df.drop_duplicates().reset_index(drop=True)

df_vendor_updated = parse_vendor_data_from_logs()
print(df_vendor_updated)


  company_code vendor_number                                 name
0         SG80    1000148216          SH COGENT LOGISTICS PTE LTD
1         SG77    1000338436         SPEEDYLINK LOGISTICS SDN BHD
2         SG78    1000447786          AIW LOGISTICS (M) SDN. BHD.
3         SG88    1000265996  AARDWOLF PESTKARE (SINGAPORE) PTE L
4         SG80    1000447786          AIW LOGISTICS (M) SDN. BHD.
5         SG79    1000152145                  HOPLITE ENGINEERING
6         SG77    1000241177   KIM ENG SENG ENGINEERING & TRADING
